In [245]:
import numpy as np 
import matplotlib.pyplot as plt
import skimage.io
from skimage.measure import regionprops, regionprops_table
import pandas as pd 
import numpy.ma as ma
import cv2
import os 

In [246]:
# Define image and masks folder
img_folder = './3seg+conjugated 8_26_corrected'
mask_folder = './masks'

df = pd.read_csv('all_info.csv')

In [247]:
def get_condition(df, condition):
    imgs_path = []
    markers = []

    df = df[(df.Condition == condition) & (df.Z_stack == 'Combined')]
    for row in df.itertuples():
        if row.Marker in ['DAPI','Phalloidin', 'WGA']:
            continue
        imgs_path.append(row.Path_corrected)
        markers.append(row.Marker)
    return imgs_path, markers

def get_masks(mask_folder):
    # Read masks
    masks = {}

    for (dirpath, dirnames, filenames) in os.walk(mask_folder):
        for name in sorted(filenames):
            if 'tiff' in name:
                condition = name.split('_')[0]
                masks[condition] = masks.get(condition, {})
                filename = os.path.join(dirpath, name)
            if 'cyto' in name:
                img = skimage.io.imread(filename)
                masks[condition]['cyto'] = img 
            elif 'nuclei' in name:
                img = skimage.io.imread(filename)
                masks[condition]['nuclei'] = img
    return masks

def measure_prop(df, masks, show=False, save=False):
    appended_data = []

    # Get each segmented cell region with corresponding nuclei and cell
    for condition in masks.keys():
        mask_cyto = masks[condition]['cyto']
        mask_nuclei = masks[condition]['nuclei']

        cell = np.zeros((mask_cyto.shape), dtype = np.uint8)
        nuclei = np.zeros((mask_cyto.shape), dtype = np.uint8)
        cyto = np.zeros((mask_cyto.shape), dtype = np.uint8)

        for label in range(1, mask_cyto.max()):
            # Check if cell has nuclei
            cell_mask = np.where(mask_cyto == label, 1, 0).astype(np.uint8)
            maski = cv2.bitwise_and(mask_nuclei, mask_nuclei, mask = cell_mask)

            # If no nuclei detected then pass
            if maski.max() == 0 :
                continue

            # Link label accross cell, nuclei, cyto
            cell = np.where(mask_cyto == label, label, cell)
            nuclei = np.where(maski > 0, label, nuclei)
            maski = cv2.subtract(cell_mask, maski)
            cyto = np.where(maski > 0, label, cyto)
            
        imgs, markers = get_condition(df, condition)
        for idx, img_path in enumerate(imgs):
            img = skimage.io.imread(img_path)
            cell_props = regionprops_table(cell, img, properties=['area', 'mean_intensity'])
            nuclei_props = regionprops_table(nuclei, img, properties=['area', 'mean_intensity'])
            cyto_props = regionprops_table(cyto, img, properties=['area', 'mean_intensity'])

            props = [cell_props, nuclei_props, cyto_props]
            prefix = ['Cell ', 'Nuclei ', 'Cyto ']
            df_marker = pd.concat([pd.DataFrame(prop).add_prefix(suffix[idx]) for idx, prop in enumerate(props)], axis = 1)
            df_marker['Marker'] = markers[idx]
            df_marker['Condition'] = condition
            df_marker['Id'] = df_marker.index
            appended_data.append(df_marker)
        if show:
            fig, ax = plt.subplots(1,3, figsize=(15,5))
            ax[0].imshow(cell)
            ax[1].imshow(nuclei)
            ax[2].imshow(cyto)
            plt.show()

    df_prop = pd.concat(appended_data, ignore_index=True)
    df_prop = df_prop.sort_values(by=['Condition','Id']).reset_index(drop=True)

    if save:
        df_prop.to_csv('measurements.csv', index=False)

    return df_prop

In [248]:
masks = get_masks(mask_folder)

In [249]:
df_prop = measure_prop(df, masks, save=True)

In [250]:
df_prop

,Cell area,Cell mean_intensity,Nuclei area,Nuclei mean_intensity,Cyto area,Cyto mean_intensity,Marker,Condition,Id
0,19790,1473.307428,966,8407.890269,18824,1117.442201,Concanavadin A,Fw1,0
1,19790,213.940879,966,300.390269,18824,209.504516,APC,Fw1,0
2,19790,87.498029,966,94.752588,18824,87.125744,Cyclin D1,Fw1,0
3,19790,261.507276,966,895.098344,18824,228.992988,Cyclin E,Fw1,0
4,19790,225.033957,966,385.269151,18824,216.811092,EMMPRIN,Fw1,0
...,...,...,...,...,...,...,...,...,...
819,19467,484.493296,3926,1284.717524,15541,282.338974,Cyclin E,Fw3,30
820,19467,377.179740,3926,762.017575,15541,279.961199,EMMPRIN,Fw3,30
821,19467,151.947090,3926,113.231279,15541,161.727559,WNT-1,Fw3,30
822,19467,472.497611,3926,1455.043046,15541,224.284924,Non-phospho-B-catenin,Fw3,30
